In [2]:
import glob
import json
import numpy as np

In [3]:
# old: "logs/2019-01-31-16-06-36" # black; "logs/2019-01-31-16-05-39" # grey
# folder = "logs/2019-01-31-19-09-50" # White
#folder_g = "logs/2019-01-31-19-10-28" # Grey
#folder_b = "logs/2019-01-31-19-10-39" # Black
#folder_b = "logs/2019-02-05-14-25-54" # Black
#folder_g = "logs/2019-02-05-14-28-02" # Grey
#folder_b = "logs/2019-02-07-13-59-59" # Problematic Black
#folder_g = "logs/2019-02-07-14-00-11" # Problematic Grey

# New without update
#folder_g = "logs/2019-02-07-18-21-49"
#folder_b = "logs/2019-02-07-18-22-15"

folder_b = "logs/2019-02-07-20-22-23"
folder_g = "logs/2019-02-07-20-22-29"

folder_output = "logs/images"
# folder_b = "logs/2019-02-01-11-56-49" # Problematic black

# color = "black"
# color = "grey"
models = ["brp.v1.prism",
          "consensus.2.prism",
          "crowds.prism",
          "csma.2-2.prism",
          "firewire.true.prism",
          "haddad-monmege.pm",
          "ij.3.v1.prism",
          "ij.10.v1.prism",          
          "leader_sync.3-2.v1.prism",
          "pacman.nm",
          "philosophers-mdp.3.v1.prism",
          "pnueli-zuck.3.v1.prism",
          "rabin.3.prism",
          "wlan.0.prism",
          "zeroconf.prism",
          "cdmsn.prism",
          "cloud_5.prism",
          "mdsm.prism",
          "team-form-3.prism"
         ]

logs = {}

for model in models:
    l_g = glob.glob(folder_g+"/"+model+"*.log")
    s_g = glob.glob(folder_g+"/"+model+"*.stat")
    l_b = glob.glob(folder_b+"/"+model+"*.log")
    s_b = glob.glob(folder_b+"/"+model+"*.stat")
    model_type = ""
    contents = open("models/"+model).read()
    if l_g and s_g and l_b and s_b:
        for t in ["smg\n", "mdp\n", "dtmc\n"]:
            if t in contents:
                #print(model+","+t.strip())
                model_type = [t.strip()]*len(l_g)
        logs[model] = list(zip(model_type, l_g, l_b, s_g, s_b))

In [4]:
import matplotlib.pyplot as plt
from matplotlib import ticker
%matplotlib inline

for model in logs:
    i = 3
    for e in logs[model]:
        i = i+1
        model_type, log_file_g, log_file_b, stat_file_g, stat_file_b = e
        with open(log_file_g) as f_g, open(log_file_b) as f_b:
            # Grey
            global_times_g = []
            precisions_g = []
            trials_g = []
            lower_bounds_g = []
            upper_bounds_g = []
            states_g = -1
            unset_g = 0
            result_g = -1
            completed_g = -1
            
            # Black
            global_times_b = []
            precisions_b = []
            trials_b = []
            lower_bounds_b = []
            upper_bounds_b = []
            states_b = -1
            result_b = -1
            completed_b = -1
            
            prop, consts, _, _ = log_file_g.split("/")[2][len(model)+1:].split("-")
            prop1, consts1, _, _ = log_file_b.split("/")[2][len(model)+1:].split("-")
            
            for line in f_g:
                if line.startswith("JSON: "):
                    cur = json.loads(line[len("JSON: "):])
                    global_times_g.append(float(cur['GlobalTimerSecs']))
                    precisions_g.append(float(cur['Precision']))
                    trials_g.append(float(cur['Trials']))
                    lower_bounds_g.append(float(cur['Value']['Lower']))
                    upper_bounds_g.append(float(cur['Value']['Upper']))
                    states_g = int(cur['StateInfos']['numStates'])
                    unset_g = int(cur['StateInfos']['numUnset'])
                if line.startswith("Model checking completed in"):
                    completed_g = float(line.split(" ")[4])
                if line.startswith("Result"):
                    result_g = float(line.split(": ")[1])
                    
            for line in f_b:
                if line.startswith("JSON: "):
                    cur = json.loads(line[len("JSON: "):])
                    global_times_b.append(float(cur['GlobalTimerSecs']))
                    precisions_b.append(float(cur['Precision']))
                    trials_b.append(float(cur['Trials']))
                    lower_bounds_b.append(float(cur['Value']['Lower']))
                    upper_bounds_b.append(float(cur['Value']['Upper']))
                    states_b = int(cur['StateInfos']['numStates'])
                if line.startswith("Model checking completed in"):
                    completed_b = float(line.split(" ")[4])
                if line.startswith("Result"):
                    result_b = float(line.split(": ")[1])
            
            if (len(global_times_g) == 0 and result_g < 0):
                print("%s;%s;%s;%9f;%9f;%s;%s;%.5f;%.5f;%.5f;%.5f" % (model, 
                                                  consts, 
                                                  prop, 
                                                  1 if len(precisions_g) == 0 else precisions_g[-1], 
                                                  1 if len(precisions_b) == 0 else precisions_b[-1], 
                                                  ((states_g-unset_g) if states_g >= 0 else ""), 
                                                  (states_b if states_b >= 0 else ""),
                                                  (0 if len(lower_bounds_g) == 0 else lower_bounds_g[-1]),
                                                  (1 if len(upper_bounds_g) == 0 else upper_bounds_g[-1]),
                                                  (0 if len(lower_bounds_b) == 0 else lower_bounds_b[-1]),
                                                  (1 if len(upper_bounds_b) == 0 else upper_bounds_b[-1])))
            else:
                plt.figure(figsize=(3.5,3.5))
                plt.style.use('seaborn')
                if len(global_times_g) == 1 and result_g >= 0:
                    plt.plot(global_times_g, lower_bounds_g, label='Lower Bound (G)', marker='o', markersize=3)
                    plt.plot(global_times_g, upper_bounds_g, label='Upper Bound (G)', marker='o', markersize=3)
                else:
                    if result_g < 0:
                        #global_times_g.append(3600)
                        global_times_g.append(1800)
                        lower_bounds_g.append(lower_bounds_g[-1])
                        upper_bounds_g.append(upper_bounds_g[-1])
                    plt.plot(global_times_g, lower_bounds_g, label='Lower Bound (G)')
                    plt.plot(global_times_g, upper_bounds_g, label='Upper Bound (G)')
                if (len(global_times_b) == 0 and result_b < 0):
                    global_times_b.append(0)
                    global_times_b.append(1800)
                    lower_bounds_b.append(0)
                    lower_bounds_b.append(0)
                    upper_bounds_b.append(1)
                    upper_bounds_b.append(1)
                    precisions_b.append(1)
                    precisions_b.append(1)
                    states_b = 0
                    plt.plot(global_times_b, lower_bounds_b, label='Lower Bound (B)', marker='o', linestyle='dashed', markersize=3)
                    plt.plot(global_times_b, upper_bounds_b, label='Upper Bound (B)', marker='o', linestyle='dashed', markersize=3)
                elif len(global_times_b) == 1 and result_b >= 0:
                    plt.plot(global_times_b, lower_bounds_b, label='Lower Bound (B)', marker='o', linestyle='dashed', markersize=3)
                    plt.plot(global_times_b, upper_bounds_b, label='Upper Bound (B)', marker='o', linestyle='dashed', markersize=3)
                else:
                    if result_b < 0:
                        #global_times_b.append(3600)
                        global_times_b.append(1800)
                        lower_bounds_b.append(lower_bounds_b[-1])
                        upper_bounds_b.append(upper_bounds_b[-1])
                    plt.plot(global_times_b, lower_bounds_b, label='Lower Bound (B)', linestyle='dashed')
                    plt.plot(global_times_b, upper_bounds_b, label='Upper Bound (B)', linestyle='dashed')
                if completed_g >= 0:
                    plt.plot(completed_g, result_g, marker='*', markersize=10, label="Result (G)")
                if completed_b >= 0:
                    plt.plot(completed_b, result_b, marker='*', markersize=10, label="Result (B)")
                ax = plt.gca()
                ax.set_yticks(np.arange(0,1.1,0.1))
                #ax.set_xscale('log', basex=2, nonposx='clip')
                #ax.set_xlim(-100, 3700)
                ax.set_xlim(-100, 1900)
                #plt.xticks([0, 300, 600, 1200, 1800, 2400, 3000, 3600], ("0", "5", "10", "20", "30", "40", "50", "60"), rotation=0)
                plt.xticks([0, 300, 600, 900, 1200, 1500, 1800], ("0", "5", "10", "15", "20", "25", "30"), rotation=0)
                #ax.get_xaxis().set_major_formatter(ticker.ScalarFormatter())
                plt.ylabel('Value')
                plt.xlabel('Time (min)')
                #plt.title(model.split(".")[0])
                plt.title("Model: " + model.split(".")[0])#+", Property: "+prop)
                plt.legend()            
                plt.grid(True)
                plt.ylim(-0.1,1.1)
                #plt.show()
                filename = model.replace(".", "-")+"-"+prop+"-"+consts+"-"+str(i)+"-full"
                #plt.savefig(folder+"/"+log_file.split("/")[2][:-4]+".png", bbox_inches='tight')
                #plt.savefig(folder_output+"/"+log_file.split("/")[2][:-4]+".pdf", bbox_inches='tight')
                plt.savefig(folder_output+"/"+filename+".pdf", bbox_inches='tight')
#                print("\includegraphics[width=0.48\\textwidth]{images/"+filename+".pdf}")
                #print("Wrote graph to " + folder+"/"+log_file.split("/")[2][:-4]+".pdf")
                print("%s;%s;%s;%9f;%9f;%s;%s;%.5f;%.5f;%.5f;%.5f" % (model, consts, prop, 
                                                  precisions_g[-1], precisions_b[-1], 
                                                  (states_g-unset_g), states_b,
                                                  lower_bounds_g[-1], upper_bounds_g[-1],
                                                  lower_bounds_b[-1], upper_bounds_b[-1]
                                                 ))
                plt.close()
        #break # Use this break to generate only the first image

brp.v1.prism;N=16,MAX=2;p1; 0.002163; 0.171647;645;614;0.00000;0.00216;0.00000;0.17165
brp.v1.prism;N=16,MAX=2;p1; 0.002446; 0.173377;645;614;0.00000;0.00245;0.00000;0.17338
brp.v1.prism;N=16,MAX=2;p1; 0.002250; 0.168370;645;614;0.00000;0.00225;0.00000;0.16837
brp.v1.prism;N=16,MAX=2;p1; 0.002234; 0.234891;645;614;0.00000;0.00223;0.00000;0.23489
brp.v1.prism;N=16,MAX=2;p1; 0.002231; 0.166774;645;614;0.00000;0.00223;0.00000;0.16677
brp.v1.prism;N=16,MAX=2;p1; 0.002239; 0.168318;645;614;0.00000;0.00224;0.00000;0.16832
brp.v1.prism;N=16,MAX=2;p1; 0.002133; 0.234911;645;614;0.00000;0.00213;0.00000;0.23491
brp.v1.prism;N=16,MAX=2;p1; 0.002306; 0.222239;645;614;0.00000;0.00231;0.00000;0.22224
consensus.2.prism;K=2;disagree; 0.009057; 0.161533;272;273;0.10392;0.11297;0.07144;0.23297
consensus.2.prism;K=2;disagree; 0.009459; 0.169216;272;273;0.10425;0.11371;0.06984;0.23905
consensus.2.prism;K=2;disagree; 0.009295; 0.161433;272;273;0.10417;0.11346;0.07138;0.23281
consensus.2.prism;K=2;disagree;

pnueli-zuck.3.v1.prism;;live; 0.000000; 0.027832;569;1921;1.00000;1.00000;0.97217;1.00000
pnueli-zuck.3.v1.prism;;live; 1.000000; 0.027510;;1909;0.00000;1.00000;0.97249;1.00000
pnueli-zuck.3.v1.prism;;live; 0.000000; 0.027508;1934;1920;1.00000;1.00000;0.97249;1.00000
pnueli-zuck.3.v1.prism;;live; 0.000000; 0.028164;374;1917;1.00000;1.00000;0.97184;1.00000
pnueli-zuck.3.v1.prism;;live; 0.000000; 0.027508;32;1910;1.00000;1.00000;0.97249;1.00000
pnueli-zuck.3.v1.prism;;live; 0.000000; 0.027196;95;1922;1.00000;1.00000;0.97280;1.00000
pnueli-zuck.3.v1.prism;;live; 0.000000; 0.027198;455;1912;1.00000;1.00000;0.97280;1.00000
rabin.3.prism;;live; 0.000000; 0.024694;1928;1089;1.00000;1.00000;0.97531;1.00000
rabin.3.prism;;live; 0.000000; 0.024610;1868;1089;1.00000;1.00000;0.97539;1.00000
rabin.3.prism;;live; 0.000000; 0.024948;1899;1089;1.00000;1.00000;0.97505;1.00000
rabin.3.prism;;live; 0.000000; 0.025078;1878;1089;1.00000;1.00000;0.97492;1.00000
rabin.3.prism;;live; 0.000000; 0.024818;1887;1

In [4]:
model = "consensus.2.prism"
glob.glob(folder+"/"+model+"*.log")

NameError: name 'folder' is not defined